Давайте теперь попробуем использовать знания этого занятия для чего-то полезного.
Часто доступные нам данные представленны не в том формате, или на их основе можно получить дополнительную информацию. Иногда для получения этой информации нам нужно отправить запросы и обработать ответы из онлайн-сервиса

Классический пример - геокодинг, те определение координат места по его адресу. Геокодинг по всему миру - крайне сложная задача, требующая сравнительно большой инфраструктуры и больших объемов постоянно обновляющейся информации. Геокодинг как услуга предоставляется компаниями типа Yandex, Google, Mapbox, Carto и другими, — а так-же организацией Open Street Maps на основе их данных. Имейте в виду - большинство сервисов имеют бесплатную квоту, но практически все - имеют лицензию, во многих случаях (Google, Yandex) очень суровую.

Давайте представим что у нас есть набор данных с адресами, и нам нужно получить их координаты. Использовать будем сервис геолокации от OSM - Nominatim.

Посмотрите на документацию сервиса: [ссылка](http://nominatim.org/release-docs/latest/api/Search/)

Теперь обсудим технические детали:
    
1. Дано: файл csv с адресами
2. Задача: Нужно записать такой-же файл, но уже с координатами. Некоторые адреса могут не распознаться!
3. Решение: использовать сервис Nominatim
    * прочитать файл и устроить проход по строкам в цикле
    * для каждой строки сформировать запрос к nominatim, получить, распарсить и сохранить результат
    * все данные сохранить в новый файл
    

In [19]:
from csv import DictReader, DictWriter  # чтение csv
import requests as rq
import time

# Чтение данных

In [2]:
path = '../data/l2/geographies.csv'

In [10]:
with open(path, mode='r') as f: # mode r -- read
    data = [row for row in DictReader(f)]

# Функция запроса к Nominatim

In [14]:
?rq.get()

Signature: rq.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary or bytes to be sent in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      ~/anaconda3/lib/python3.6/site-packages/requests/api.py
Type:      function


In [20]:
def geocode(address:str, sleep=1, **kwargs):
    '''geocode using nomitatim service
    
    docs: http://nominatim.org/release-docs/latest/api/Search/
    
    args:
        address(str): address to query'''
    url = 'https://nominatim.openstreetmap.org/search'
    params = {'q':address, 'format':'geojson'}.update(kwargs)

    r = rq.get(url, params=params)
    r.raise_for_status()  # raise if something wrong with response
    time.sleep(sleep) # dodge nominatim's requests-per-second limits
    
    return r.json()  # requests knows how to parse json answer into dictionary 
    
    
    

обратите внимание, что nominatim может фильтравать адреса для конкретного города/страны - это удобно, если ваши адреса находятся в одном городе

In [24]:
r = geocode('130 5th avenue, New York, NY, 10011, USA')
r

{'features': [{'bbox': [-73.9918909, 40.7386806, -73.9917909, 40.7387806],
   'geometry': {'coordinates': [-73.9918409, 40.7387306], 'type': 'Point'},
   'properties': {'category': 'place',
    'display_name': '130, 5th Avenue, Flatiron District, Manhattan, Manhattan Community Board 5, New York County, NYC, New York, 10011, USA',
    'importance': 0.831,
    'osm_id': '2703043812',
    'osm_type': 'node',
    'place_id': '28682016',
    'place_rank': '30',
    'type': 'house'},
   'type': 'Feature'}],
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'type': 'FeatureCollection'}

Запрос отдает массив фич (точек)ю Для простоты будем считать что первая точка всегда верная, смотрим только на нее. Но нужно предусмотреть и вариант с 0 фич!

In [25]:
def _pull_coords(response):
    if len(response['features']) == 0:
        return None
    
    return response['features'][0]['geometry']['coordinates']

# Построим цикл

In [30]:
def _geocode_data(data):
    new_data  = data.copy()
    
    for row in data:
        r = geocode(row['city'])
        coords = _pull_coords(r)

        row['lat'], row['lon'] = coords
    return data

In [32]:
result = _geocode_data(data)

# Сохраним файл

In [35]:
out_path = 'geolocated_capitals.csv'

with open(out_path, 'w') as csvfile:
        writer = DictWriter(csvfile, fieldnames=result[0].keys())
        writer.writeheader()
        
        for data in result:
            writer.writerow(data)